In [3]:
import requests
import json
import os
from bs4 import BeautifulSoup

In [77]:
class BookingWebsites:
    
    def __init__(self):
        
        self.websites = set()
        self.urls = {'muslimtravelgirl.com': 'https://muslimtravelgirl.com/comprehensive-list-of-hotel-booking-sites/',
                    'touropia.com': 'https://www.touropia.com/best-hotel-websites/',
                    'smartertravel.com': 'https://www.smartertravel.com/the-10-best-hotel-booking-sites/',
                    'hostessmeetsworld.com': 'http://www.hostessmeetsworld.com/top-10-homestay-and-lodging/'}
    
    def get(self):
        
        for s in self.urls:
                
            print(f'collecting data from {s}...')
            
            r = requests.get(self.urls[s]).text
            soup = BeautifulSoup(r, 'html.parser')
                
            if s == 'muslimtravelgirl.com':
        
                # visit each section corresponsing to the firs letter of airline's name
                for d in soup.find_all('ul'):
                    for l in d.children:
                        a_ = l.find('a')
                        if 'muslimtravelgirl' in a_.get('href'):
                            continue
                        self.websites.add(a_.text.lower().strip())
                        
            elif s == 'touropia.com':
                
                for d in soup.find_all('p'):
                    for a_ in d.find_all('a', attrs={'rel': 'nofollow'}):
                        if 'touropia.com' not in a_.get('href'):
                            self.websites.add(a_.get('href').lower().split('www.')[-1].replace('/','').strip())
            elif s == 'smartertravel.com':
                
                for p in soup.find_all('p'):
                    if 'frommer' in p.text.lower():
                        for s in p.next_siblings:
                            if s.name == 'ol':
                                for l in s.find_all('li'):
                                    self.websites.add(l.text.lower().strip())
                                    
            elif s == 'hostessmeetsworld.com':
                
                for h in soup.find_all('h1'):
                    if 'top 10 list' in h.text.lower():
                        for s in h.next_siblings:
                            a_ = s.find('a')
                            if a_:
                                url_ = a_.get('href')
                                if ('hostessmeetsworld' not in url_):
                                    self.websites.add(url_.lower().split('www.')[-1].split('/')[0].strip())
                    
        # postprocessing
        self.websites = {s for s in self.websites if '.' in s}   
                
        print(f'collected {len(self.websites)} websites')
        
        return self
    
    def save(self, file='booking_websites.txt'):
        
        if not os.path.exists('data'):
            os.mkdir('data')
            
        with open(os.path.join('data', file), 'w') as f:
            for website in self.websites:
                f.write(f'{website}\n')

In [78]:
if __name__ == '__main__':
    
    a = BookingWebsites()
    a.get().save()

collecting data from muslimtravelgirl.com...
collecting data from touropia.com...
collecting data from smartertravel.com...
collecting data from hostessmeetsworld.com...
collected 42 websites
